In [47]:
#This script uses python 3.12 to load data from a decrypted transaction into the table "transaction_detail_msg_send" from the postgres database "blockchain"
#Libraries used os, json, psycopg2

#Connect to database
import os
import json
import psycopg2

def connect_to_database():
    # Replace with your actual database connection parameters
    db_params = {
        "dbname": "blockchain",
        "user": "postgres",
        "password": "auth2020",
        "host": "localhost",
        "port": "5432"
    }

    try:
        conn = psycopg2.connect(**db_params)
        return conn
    except psycopg2.Error as e: #Modify to send info to log life
        print(f"Error connecting to the database: {e}")
        return None

In [48]:
#Grab FK "Transaction_send_Id from table transaction_header_send"
def get_most_recent_transaction_send_id(conn):
    # Replace with your actual query to get the most recent transaction_send_id
    # Example: SELECT MAX(transaction_send_id) FROM your_table_name;
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT transaction_send_id FROM transaction_header_send ORDER BY transaction_send_id DESC LIMIT 1;")
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                return None
    except psycopg2.Error as e:
        print(f"Error fetching data: {e}")
        return None


In [49]:
#load fk and data from json to msg table
def main():
    block = '6041182' # Insert desired block number
    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

    try:
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                conn = connect_to_database()
                transaction_send_id = get_most_recent_transaction_send_id(conn)

                if transaction_send_id:
                    insert_queries = [] #Loop through all messages, so that each message as its own row in table

                    for message in json_data['tx']['body']['messages']: 
                        sender = message["from_address"]
                        receiver = message["to_address"]
                        #SQL Insert statement for FK and the data that was extracted from json file
                        msg_insert_query = f"""
                            INSERT INTO transaction_detail_msg_send (transaction_send_id, sender, receiver)
                            VALUES ('{transaction_send_id}', '{sender}', '{receiver}');
                        """
                        insert_queries.append(msg_insert_query)

                    with conn.cursor() as cursor:
                        for query in insert_queries:
                            cursor.execute(query)
                        conn.commit()

                    print("Data inserted successfully!")
                else:
                    print("No recent transaction_send_id found.")
        else:
            print(f"JSON file not found: {file_path}")
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")

if __name__ == "__main__":
    main()


Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/6041182.json
Data inserted successfully!
